In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:00<00:00, 37.3MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 962kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 18.0MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 24.5MB/s]


In [11]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape) # N=batch size, C=channels, H=height, W=width
    print("Shape of y: ", y.shape, y.dtype)
    break

test_dataloader

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [15]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # [64, 1, 28, 28] -> [64, 1*28*28]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
for batch, (X, y) in enumerate(train_dataloader):

    print(batch, X.shape, y.shape)

0 torch.Size([64, 1, 28, 28]) torch.Size([64])
1 torch.Size([64, 1, 28, 28]) torch.Size([64])
2 torch.Size([64, 1, 28, 28]) torch.Size([64])
3 torch.Size([64, 1, 28, 28]) torch.Size([64])
4 torch.Size([64, 1, 28, 28]) torch.Size([64])
5 torch.Size([64, 1, 28, 28]) torch.Size([64])
6 torch.Size([64, 1, 28, 28]) torch.Size([64])
7 torch.Size([64, 1, 28, 28]) torch.Size([64])
8 torch.Size([64, 1, 28, 28]) torch.Size([64])
9 torch.Size([64, 1, 28, 28]) torch.Size([64])
10 torch.Size([64, 1, 28, 28]) torch.Size([64])
11 torch.Size([64, 1, 28, 28]) torch.Size([64])
12 torch.Size([64, 1, 28, 28]) torch.Size([64])
13 torch.Size([64, 1, 28, 28]) torch.Size([64])
14 torch.Size([64, 1, 28, 28]) torch.Size([64])
15 torch.Size([64, 1, 28, 28]) torch.Size([64])
16 torch.Size([64, 1, 28, 28]) torch.Size([64])
17 torch.Size([64, 1, 28, 28]) torch.Size([64])
18 torch.Size([64, 1, 28, 28]) torch.Size([64])
19 torch.Size([64, 1, 28, 28]) torch.Size([64])
20 torch.Size([64, 1, 28, 28]) torch.Size([64])
21

In [23]:
# Optimising Model Parameters
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_func, optimiser):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_func(pred, y)

        # Backpropagation
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [27]:
def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_func, optimiser)
    test(test_dataloader, model, loss_func)
print("Done!")

Epoch 1
-------------------------------
loss: 2.158488 [    0/60000]
loss: 2.147775 [ 6400/60000]
loss: 2.082053 [12800/60000]
loss: 2.105493 [19200/60000]
loss: 2.048512 [25600/60000]
loss: 1.979381 [32000/60000]
loss: 2.017433 [38400/60000]
loss: 1.927745 [44800/60000]
loss: 1.942948 [51200/60000]
loss: 1.867725 [57600/60000]
Test Error: 
 Accuracy: 55.2%, Avg loss: 1.861982 

Epoch 2
-------------------------------
loss: 1.893631 [    0/60000]
loss: 1.862122 [ 6400/60000]
loss: 1.736936 [12800/60000]
loss: 1.794779 [19200/60000]
loss: 1.670643 [25600/60000]
loss: 1.622666 [32000/60000]
loss: 1.657799 [38400/60000]
loss: 1.548615 [44800/60000]
loss: 1.585489 [51200/60000]
loss: 1.483353 [57600/60000]
Test Error: 
 Accuracy: 61.6%, Avg loss: 1.495510 

Epoch 3
-------------------------------
loss: 1.559708 [    0/60000]
loss: 1.526900 [ 6400/60000]
loss: 1.372764 [12800/60000]
loss: 1.461000 [19200/60000]
loss: 1.329319 [25600/60000]
loss: 1.329317 [32000/60000]
loss: 1.348968 [38400/

In [29]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0323,  0.0159,  0.0145,  ...,  0.0129,  0.0132, -0.0275],
                      [ 0.0295,  0.0294,  0.0288,  ...,  0.0116,  0.0018, -0.0111],
                      [ 0.0248, -0.0081, -0.0078,  ...,  0.0172,  0.0255, -0.0232],
                      ...,
                      [ 0.0041, -0.0168, -0.0172,  ..., -0.0336, -0.0088, -0.0010],
                      [-0.0125,  0.0091, -0.0115,  ..., -0.0243, -0.0023, -0.0015],
                      [-0.0278,  0.0335, -0.0010,  ..., -0.0220, -0.0186,  0.0316]],
                     device='mps:0')),
             ('linear_relu_stack.0.bias',
              tensor([-3.4659e-02,  1.8358e-03, -1.8861e-02,  2.0970e-02,  1.9491e-02,
                      -2.1207e-02,  7.6103e-03,  2.2209e-02,  4.3259e-04,  3.1322e-03,
                      -6.1687e-04,  2.0099e-02, -8.5793e-03,  3.0769e-02,  2.3239e-02,
                      -5.2902e-03,  1.4201e-02, -1.2353e-02,  4.5747e-04, -2.7377e

In [30]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [31]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [33]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
